In [1]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
nltk.download('punkt')

True

In [9]:
data = pd.read_csv('segmented_answer.csv', delimiter=',', header=None)[0]
data = [i for i in data if len(i) > 0]

In [10]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [11]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(alpha=alpha, 
                min_alpha=0.00025)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=1)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

In [51]:
def cosine_similarity(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [62]:
correct = 0
test_dataset = pd.read_csv('cnn_data.csv', delimiter=',')
vectorized = pd.DataFrame()
vectorized[0] = test_dataset['0'].apply(lambda x : model.infer_vector(x.split(' ')))
vectorized[1] = test_dataset['1'].apply(lambda x : model.infer_vector(x.split(' ')))

In [67]:
for i in range(len(col1)):
    sim = int(cosine_similarity(vectorized[0][i], vectorized[1][i]) > 0.8)
    correct += int(sim == test_dataset['2'][i])

In [68]:
correct / 24000

0.82675